In [1]:
import requests
import json
import pandas as pd
from pandas.core.frame import DataFrame
import csv
import time
import datetime

In [2]:
s = requests.Session()

In [3]:
start_headers = {
"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"accept-encoding": "gzip, deflate, br",
"accept-language": "en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7",
"dnt": "1",
"sec-fetch-dest": "document",
"sec-fetch-mode": "navigate",
"sec-fetch-site": "none",
"upgrade-insecure-requests": "1",
"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36",
}

In [4]:
resp = s.get(
    "https://www.uber.com/us/en/price-estimate/",
    headers=start_headers,
)
assert resp.status_code == 200


In [5]:
url = 'https://www.uber.com/api/loadFEEstimates'

In [6]:
# Grand central station, 89 E 42nd St, New York, NY
# World Trade Center, New York, NY
# JFK airport
# Citi Field in Queens

#{"id": "ChIJ-b2RmVlZwokRpb1pwEQjss0","latitude":40.75285290000001,"locale": "en","longitude": -73.97767550000002,"provider": "google_places"} 
#{"id":"ChIJ-4MbiBlawokR5ixJ4E8Km5c","provider":"google_places", "locale":"en","latitude":40.7118011,"longitude":-74.0131196 } 
#{"id": "ChIJR0lA1VBmwokR8BGfSBOyT-w","latitude": 40.64131109999999,"locale": "en","longitude": -73.77813909999999,"provider": "google_places"} 
#{"id": "ChIJldFHseBfwokRO3aZ8NSVW_U","latitude": 40.7570877,"locale": "en","longitude": -73.8458213,"provider": "google_places"} 

In [7]:
# need to change,
# since you have static origin and destination, you just grab it from chrome Inspect
payload1 = '''{"origin":{"id": "ChIJ-b2RmVlZwokRpb1pwEQjss0","latitude":40.75285290000001,"locale": "en","longitude": -73.97767550000002,"provider": "google_places"} ,"destination":{"id":"ChIJ-4MbiBlawokR5ixJ4E8Km5c","provider":"google_places", "locale":"en","latitude":40.7118011,"longitude":-74.0131196 } ,"locale":"en"} '''
payload2 = '''{"origin":{"id": "ChIJ-b2RmVlZwokRpb1pwEQjss0","latitude":40.75285290000001,"locale": "en","longitude": -73.97767550000002,"provider": "google_places"} ,"destination":{"id": "ChIJR0lA1VBmwokR8BGfSBOyT-w","latitude": 40.64131109999999,"locale": "en","longitude": -73.77813909999999,"provider": "google_places"} ,"locale":"en"}'''
payload3 = '''{"origin":{"id": "ChIJ-b2RmVlZwokRpb1pwEQjss0","latitude":40.75285290000001,"locale": "en","longitude": -73.97767550000002,"provider": "google_places"} ,"destination":{"id": "ChIJldFHseBfwokRO3aZ8NSVW_U","latitude": 40.7570877,"locale": "en","longitude": -73.8458213,"provider": "google_places"} ,"locale":"en"} '''


In [8]:
extra_headers = {
    "content-length": "293",
    "content-type": "application/json",
    "x-csrf-token": "x",
    "origin": "https://www.uber.com",
    "referer": "https://www.uber.com/us/en/price-estimate/",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
}

In [9]:
resp1 = s.post(url, data=payload1, headers={**s.headers, **extra_headers})
assert resp1.status_code == 200
resp2 = s.post(url, data=payload2, headers={**s.headers, **extra_headers})
assert resp2.status_code == 200
resp3 = s.post(url, data=payload3, headers={**s.headers, **extra_headers})
assert resp3.status_code == 200

In [10]:
dic1=(json.loads(resp1.content))
dic2=(json.loads(resp2.content))
dic3=(json.loads(resp3.content))
list1 = dic1['data']['prices']
list2 = dic2['data']['prices']
list3 = dic3['data']['prices']
option=['Pool - Unavailable','UberX','WAV','UberXL','Car Seat','Black','Black SUV']
time_stamp = datetime.datetime.now().strftime('%Y.%m.%d-%H:%M')
time = [time_stamp]*7

In [11]:
def option_price(l):
    op=[]
    for i in range(7):
        op.append(l[i]['fareString'])
    return op

In [12]:
WTC= option_price(list1)
JFK= option_price(list2)
CITI= option_price(list3)

In [13]:
summ = {'Time':time, 'Options':option,'GCS-WTC':WTC,'GCS-JFK':JFK,'GCS-CITI':CITI} 
df1=pd.DataFrame(summ)

In [14]:
df=pd.read_csv('/Users/zhaoyijie/Desktop/tasks/Uber.csv')
df=df.append(df1)

In [15]:
df.to_csv('/Users/zhaoyijie/Desktop/tasks/Uber.csv',index=False,header=True)

In [16]:
# use crontab to realized update
# 0 8,12,18,23 * * * /Applications/anaconda3/bin/python3 /Users/zhaoyijie/Desktop/tasks/uber_scrape.py >> /Users/zhaoyijie/Desktop/tasks/cron.log 2>&1

In [17]:
df1

,Time,Options,GCS-WTC,GCS-JFK,GCS-CITI
0,2020.06.11-18:00,Pool - Unavailable,$22.75,$54.14,$43.44
1,2020.06.11-18:00,UberX,$26.38,$56.75,$48.08
2,2020.06.11-18:00,WAV,$26.38,$56.75,$48.08
3,2020.06.11-18:00,UberXL,$29.00,$67.89,$55.08
4,2020.06.11-18:00,Car Seat,$37.52,$79.47,$59.22
5,2020.06.11-18:00,Black,$44.13,$107.19,$75.83
6,2020.06.11-18:00,Black SUV,$64.83,$152.60,$107.82
